# Create a retreat mask from basins and Calfin

In [1]:
import geopandas as gp
gp.options.io_engine = "pyogrio"
import pandas as pd
from tqdm.auto import tqdm
import xarray as xr
from geocube.api.core import make_geocube
import numpy as np
import rioxarray as rxr
from joblib import Parallel, delayed
from pathlib import Path
from typing import Union

In [2]:
imbie = gp.read_file("/Users/andy/Google Drive/My Drive/data/gris-outline/gris-outline-imbie-1980.shp").to_crs("EPSG:3413")
calfin = gp.read_file("/Users/andy/Google Drive/My Drive/data/Calfin/level-1_shapefiles-greenland-termini-closed/termini_1980-2019_Greenland_closed_valid_v1.gpkg")

/Users/andy/miniforge3/envs/pism-ragis/lib/python3.11/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)


In [3]:
bm = rxr.open_rasterio("/Users/andy/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc")

In [4]:
calfin_dissolved = calfin.dissolve()

In [5]:
imbie_union = imbie.union(calfin_dissolved)

In [6]:
x_min = -653000
x_max = 879700
y_min = -632750
y_max = -3384350
bbox = [x_min, y_min, x_max, y_max]
geom = {"type": "Polygon", "crs": {"properties": {"name": "EPSG:3413"}}, "bbox": bbox, 
        "coordinates": [[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]}

In [19]:
def create_ds(ds1, ds2, mon, crs: str = "EPSG:3413", like: Union[None, xr.Dataset] = None):
    if len(ds1) > 0:
        diff = ds2.difference(ds1.dissolve().buffer(5))
        n = len(diff)
        diff_df = {"land_ice_area_fraction_retreat": np.ones(n)}
        diff_gp = gp.GeoDataFrame(data=diff_df, geometry=diff, crs=crs)
        ds = make_geocube(vector_data=diff_gp, geom=geom, like=bm)
        ds = ds.expand_dims(time=[mon])
        fn = Path(f"frontretreat_{mon.year}-{mon.month}.nc", encoding={"zlib": True, "complevel": 2})
        print(ds)
        ds.to_netcdf(fn)
        del ds
        return fn

In [8]:
calfin_ds = calfin.set_index("QDate").groupby(by=pd.Grouper(freq='ME'))
nt = len(calfin_ds)


In [ ]:
n_jobs = 6
result = Parallel(n_jobs=n_jobs)(
    delayed(create_ds)(ds,
        imbie_union, mon, like=bm)
    for mon, ds in tqdm(calfin_ds))

  0%|          | 0/466 [00:00<?, ?it/s]

In [ ]:
ds = xr.open_mfdataset(result)